In [1]:
import mysqlx, os

session = mysqlx.get_session({
    'host': 'localhost',
    'port': 33080,              # if Classic is using port 3308
    'user': 'root',
    'password': os.environ.get('MYSQL_PWD')
})

In [2]:
# create schema+collection before, w/ mysqlsh
db = session.get_schema('store')
coll = db.get_collection('books')
coll.count()

0

In [3]:
import json

with open('../../data/store.json') as f:
    books = json.load(f)['store']['book']
    print(json.dumps(books, indent=4))

[
    {
        "category": "reference",
        "author": "Nigel Rees",
        "title": "Sayings of the Century",
        "price": 8.95,
        "editions": [
            1993,
            1995,
            1998
        ]
    },
    {
        "category": "fiction",
        "author": "Evelyn Waugh",
        "title": "Sword of Honour",
        "price": 12.99,
        "editions": [
            1996,
            1998
        ]
    },
    {
        "category": "fiction",
        "author": "Herman Melville",
        "title": "Moby Dick",
        "isbn": "0-553-21311-3",
        "price": 8.99,
        "editions": [
            1991,
            1996,
            1998
        ]
    },
    {
        "category": "fiction",
        "author": "J. R. R. Tolkien",
        "title": "The Lord of the Rings",
        "isbn": "0-395-19395-8",
        "price": 22.99
    },
    {
        "category": "software",
        "authors": [
            {
                "author": "Thomas Linn"
            },
    

In [4]:
for book in books:
    print(book)
    coll.add(book).execute()

{'category': 'reference', 'author': 'Nigel Rees', 'title': 'Sayings of the Century', 'price': 8.95, 'editions': [1993, 1995, 1998]}
{'category': 'fiction', 'author': 'Evelyn Waugh', 'title': 'Sword of Honour', 'price': 12.99, 'editions': [1996, 1998]}
{'category': 'fiction', 'author': 'Herman Melville', 'title': 'Moby Dick', 'isbn': '0-553-21311-3', 'price': 8.99, 'editions': [1991, 1996, 1998]}
{'category': 'fiction', 'author': 'J. R. R. Tolkien', 'title': 'The Lord of the Rings', 'isbn': '0-395-19395-8', 'price': 22.99}
{'category': 'software', 'authors': [{'author': 'Thomas Linn'}, {'author': 'Jane Mathews'}], 'title': 'Snowflake for Beginners', 'price': 12.49, 'editions': [1996]}


In [5]:
docs = coll.find().execute()
doc = docs.fetch_one()
while doc:
    print(doc)
    doc = docs.fetch_one()

{"_id": "000067dabd840000000000000006", "price": 8.95, "title": "Sayings of the Century", "author": "Nigel Rees", "category": "reference", "editions": [1993, 1995, 1998]}
{"_id": "000067dabd840000000000000007", "price": 12.99, "title": "Sword of Honour", "author": "Evelyn Waugh", "category": "fiction", "editions": [1996, 1998]}
{"_id": "000067dabd840000000000000008", "isbn": "0-553-21311-3", "price": 8.99, "title": "Moby Dick", "author": "Herman Melville", "category": "fiction", "editions": [1991, 1996, 1998]}
{"_id": "000067dabd840000000000000009", "isbn": "0-395-19395-8", "price": 22.99, "title": "The Lord of the Rings", "author": "J. R. R. Tolkien", "category": "fiction"}
{"_id": "000067dabd84000000000000000a", "price": 12.49, "title": "Snowflake for Beginners", "authors": [{"author": "Thomas Linn"}, {"author": "Jane Mathews"}], "category": "software", "editions": [1996]}


In [6]:
docs = (coll
    .find('title like :param')
    .limit(1)
    .bind('param', 'Sword of Honour')
    .execute())
doc = docs.fetch_one()
print(doc)

{"_id": "000067dabd840000000000000007", "price": 12.99, "title": "Sword of Honour", "author": "Evelyn Waugh", "category": "fiction", "editions": [1996, 1998]}


In [7]:
coll.remove(True).execute()
coll.count()

0

In [8]:
session.close()